In [1]:
import os
import torch
import torch.nn as nn
from PIL import Image
from torchvision import models

In [2]:
dataset_dir = r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\Dataset\Dataset-used"
train_dir = "C:\\Users\\GIGABYTE\\PycharmProjects\\Paper-FAS\\Dataset\\Dataset-used\\train"
val_dir = "C:\\Users\\GIGABYTE\\PycharmProjects\\Paper-FAS\\Dataset\\Dataset-used\\valid"
test_dir = "C:\\Users\\GIGABYTE\\PycharmProjects\\Paper-FAS\\Dataset\\Dataset-used\\test"

In [3]:
class DeepTreeModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DeepTreeModule, self).__init__()
        # Nh?nh 1: S? d?ng kernel size 3 (t?p trung v?o texture)
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        # Nh?nh 2: S? d?ng kernel size 5 (t?p trung v?o h?nh d?ng)
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=5, padding=2),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        # H?p nh?t ??u ra c?a c?c nh?nh
        self.fc = nn.Linear(out_channels * 2, out_channels)

    def forward(self, x):
        # x: [B, C, H, W]
        b1 = self.branch1(x)  # [B, out_channels, 1, 1]
        b2 = self.branch2(x)  # [B, out_channels, 1, 1]
        # Flatten c?c ??u ra
        b1 = b1.view(x.size(0), -1)
        b2 = b2.view(x.size(0), -1)
        # N?i ch?p theo chi?u k?nh
        combined = torch.cat([b1, b2], dim=1)
        out = self.fc(combined)
        return out

In [4]:
class PersonalizationModule(nn.Module):
    def __init__(self, feature_dim, personalized_dim):
        super(PersonalizationModule, self).__init__()
        # M?t m?ng fully-connected ??n gi?n ?? ?i?u ch?nh ??c tr?ng
        self.fc1 = nn.Linear(feature_dim, personalized_dim)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(personalized_dim, feature_dim)

    def forward(self, x):
        identity = x  # l?u l?i ??c tr?ng ban ??u
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        # Skip connection gi?p gi? l?i ??c tr?ng g?c
        out = out + identity
        return out

In [5]:
class FaceAntiSpoofingModel(nn.Module):
    def __init__(self, num_classes=2):
        super(FaceAntiSpoofingModel, self).__init__()
        # S? d?ng MobileNetV3 l?m backbone (pretrained)
        mobilenet = models.mobilenet_v3_small(pretrained=True)
        # L?y ph?n features c?a MobileNetV3 (lo?i b? classifier)
        self.backbone = mobilenet.features
        # K?ch th??c ??u ra c?a MobileNetV3_small th??ng l? 576 channels
        backbone_out_channels = 576

        # Module Deep Tree Learning
        deep_tree_out_channels = 256  # c? th? t?y ch?nh
        self.deep_tree = DeepTreeModule(in_channels=backbone_out_channels,
                                        out_channels=deep_tree_out_channels)

        # Module c? nh?n h?a: t? ??c tr?ng deep tree ??n kh?ng gian c? nh?n h?a
        personalized_dim = 128
        self.personalization = PersonalizationModule(feature_dim=deep_tree_out_channels,
                                                     personalized_dim=personalized_dim)

        # L?p ph?n lo?i cu?i c?ng
        self.classifier = nn.Linear(deep_tree_out_channels, num_classes)

    def forward(self, x):
        # x: [B, 3, H, W]
        features = self.backbone(x)   # [B, 576, H', W']
        tree_features = self.deep_tree(features)  # [B, 256]
        # ?p d?ng module c? nh?n h?a
        personalized_features = self.personalization(tree_features)  # [B, 256]
        logits = self.classifier(personalized_features)  # [B, num_classes]
        return logits

In [6]:
from torch.utils.data import Dataset

class FaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        :param root_dir: ???ng d?n ??n th? m?c g?c (vd: train_dir ho?c val_dir).
        :param transform: C?c h?m ti?n x? l? ?nh (Resize, Normalize, Augmentation, etc.)
        """
        self.root_dir = root_dir
        self.transform = transform
        self.data = []
        self.labels = []

        # L?n l??t duy?t qua c?c l?p (folder fake v? real)
        for label, folder_name in enumerate(['fake', 'real']):  # 0: fake, 1: real
            folder_path = os.path.join(root_dir, folder_name)
            if not os.path.exists(folder_path):
                print(f"Warning: Folder {folder_path} not found!")
                continue

            # L?y danh s?ch c?c ?nh trong t?ng th? m?c
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                self.data.append(file_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # ??c ?nh t? ???ng d?n l?u tr?
        img_path = self.data[idx]
        label = self.labels[idx]

        # M? v? x? l? ?nh
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)

        return img, label


In [24]:
from torch.utils.data import DataLoader
from torchvision import transforms

# B? ti?n x? l? ?nh (resize, normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize ?nh v? k?ch th??c ph? h?p
    transforms.ToTensor(),  # Chuy?n ??i th?nh d?ng tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ??nh ngh?a t?p train_convert v? val
train_dataset = FaceDataset(root_dir=train_dir, transform=transform)
val_dataset = FaceDataset(root_dir=val_dir, transform=transform)

# T?o DataLoader cho t?p train_convert v? val
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
import torch
from FaceAntiSpoofingModel import FaceAntiSpoofingModel

# Bước 1: Tạo instance của mô hình
model_q = FaceAntiSpoofingModel(num_classes=2)

# Bước 2: Tải state_dict đã lưu (mô hình đã lượng tử hóa)
state_dict = torch.load("face_antispoofing_model_quantized.pth", map_location=torch.device("cpu"))
model_q.load_state_dict(state_dict)

# Bước 3: Chuyển mô hình sang chế độ eval và về CPU
model_q.eval()
model_q.to("cpu")

# Bước 4: Xuất sang ONNX
dummy_input = torch.randn(1, 3, 224, 224)  # Tạo một input mẫu
torch.onnx.export(
    model_q,
    dummy_input,
    "face_antispoofing_quantized.onnx",
    export_params=True,
    opset_version=12,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}
)

print("Mô hình đã được chuyển đổi sang ONNX và lưu tại 'face_antispoofing_quantized.onnx'")

In [ ]:
import onnxruntime
def calculate_acer_onnx(onnx_session, test_loader, device):
    """
    T?nh to?n ACER, APCER, v? BPCER d?ng m? h?nh ONNX
    :param onnx_session: Inference Session c?a ONNX Runtime
    :param test_loader: DataLoader cho t?p test
    :param device: "cpu" ho?c "cuda" (GPU)
    :return: ACER, APCER, BPCER
    """
    total_real = 0
    total_fake = 0
    misclassified_real = 0
    misclassified_fake = 0

    # ??t m? h?nh sang ch? ?? ??nh gi?
    with torch.no_grad():
        for images, labels in test_loader:
            # Chuy?n d? li?u sang ??ng device
            images, labels = images.to(device), labels.to(device)

            # Chuy?n tensor images sang numpy v? ONNX y?u c?u ??u v?o ki?u numpy
            numpy_images = images.cpu().numpy()

            # Ch?y inferencing v?i m? h?nh ONNX
            output = onnx_session.run(None, {onnx_session.get_inputs()[0].name: numpy_images})
            predictions = np.argmax(output[0], axis=1)

            # Ph?n lo?i c?c tr??ng h?p
            for label, prediction in zip(labels.cpu().numpy(), predictions):
                if label == 0:
                    total_fake += 1
                    if prediction != 0:
                        misclassified_fake += 1
                elif label == 1:
                    total_real += 1
                    if prediction != 1:
                        misclassified_real += 1

    # T?nh APCER v? BPCER
    apcer = misclassified_fake / total_fake if total_fake > 0 else 0
    bpcer = misclassified_real / total_real if total_real > 0 else 0

    # T?nh ACER
    acer = (apcer + bpcer) / 2

    return acer, apcer, bpcer

import numpy as np

onnx_session = onnxruntime.InferenceSession("C:\\Users\\GIGABYTE\\PycharmProjects\\Paper-FAS\\Model\\face_antispoofing.onnx",
                                            providers=["CUDAExecutionProvider", "CPUExecutionProvider"])

device = "cuda" if torch.cuda.is_available() else "cpu"


acer, apcer, bpcer = calculate_acer_onnx(onnx_session, test_loader, device)

print(f"ACER: {acer:.4f}")
print(f"APCER: {apcer:.4f}")
print(f"BPCER: {bpcer:.4f}")


 ## Su dung Dataset cho representative dataset

In [15]:
import os
import torch
import numpy as np
from torchvision import transforms
from PIL import Image

# Định nghĩa pipeline augmentation cho representative dataset
rep_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomResizedCrop(size=224, scale=(0.9, 1.0)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x * 255)
])

# Đường dẫn representative dataset (có thể có các thư mục con như 'fake', 'real')
representative_dir = r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\Dataset\Dataset-used\valid"

# Lấy danh sách tất cả các file ảnh
image_paths = []
for root, _, files in os.walk(representative_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_paths.append(os.path.join(root, file))

print(f"Tổng số ảnh trong representative dataset: {len(image_paths)}\n")

# Load tất cả ảnh và lưu vào danh sách
all_images = []
for img_path in image_paths:
    try:
        img = Image.open(img_path).convert('RGB')
        tensor = rep_transform(img)  # Kích thước: (3, 224, 224), giá trị [0,255]
        all_images.append(tensor)
    except Exception as e:
        print(f"Lỗi khi load {img_path}: {e}")

if len(all_images) == 0:
    raise ValueError("Không load được ảnh hợp lệ từ representative dataset!")

# Stack các tensor thành một tensor lớn: (N, 3, 224, 224)
all_images_tensor = torch.stack(all_images)

# Tính toán mean và std theo từng kênh (RGB) trên toàn bộ dataset
mean = torch.mean(all_images_tensor, dim=[0, 2, 3])
std = torch.std(all_images_tensor, dim=[0, 2, 3])

mean_np = mean.numpy().tolist()
std_np = std.numpy().tolist()

print("Mean (sau khi nhân 255):", mean_np)
print("Std (sau khi nhân 255):", std_np)


Tổng số ảnh trong representative dataset: 4226

Mean (sau khi nhân 255): [140.64718627929688, 114.01417541503906, 103.77151489257812]
Std (sau khi nhân 255): [62.949188232421875, 57.36742401123047, 53.96285629272461]


In [17]:
import sys
# Thêm đường dẫn đến thư mục onnx2tflite (nếu chưa cài đặt qua pip)
sys.path.append(r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\onnx2tflite")

from onnx2tflite.converter import onnx_converter

onnx_converter(
    onnx_model_path = r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\model2\face_antispoofing_v2.onnx",
    need_simplify = True,
    output_path = r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\model2",
    target_formats = 'tflite',  # chuyển sang TFLite
    weight_quant = False,
    int8_model = True,
    int8_mean = mean_np,
    int8_std = std_np,
    image_root = representative_dir
)

Checking 0/1...


INFO:Quantization DataLoder ::Found 4226 images: 2113 fake, 2113 real
INFO:Quantization DataLoder ::Sample fake images: ['C:\\Users\\GIGABYTE\\PycharmProjects\\Paper-FAS\\Dataset\\Dataset-used\\valid\\fake\\axon2p1219_silicon_54281.jpg', 'C:\\Users\\GIGABYTE\\PycharmProjects\\Paper-FAS\\Dataset\\Dataset-used\\valid\\fake\\axon2p122_silicon_280198.jpg', 'C:\\Users\\GIGABYTE\\PycharmProjects\\Paper-FAS\\Dataset\\Dataset-used\\valid\\fake\\axon2p1232_silicon_13695.jpg', 'C:\\Users\\GIGABYTE\\PycharmProjects\\Paper-FAS\\Dataset\\Dataset-used\\valid\\fake\\axon2p1344_silicon_462893.jpg', 'C:\\Users\\GIGABYTE\\PycharmProjects\\Paper-FAS\\Dataset\\Dataset-used\\valid\\fake\\axon2p2009_silicon_2342880.jpg']
INFO:Quantization DataLoder ::Sample real images: ['C:\\Users\\GIGABYTE\\PycharmProjects\\Paper-FAS\\Dataset\\Dataset-used\\valid\\real\\asian2_real_video_36.jpg', 'C:\\Users\\GIGABYTE\\PycharmProjects\\Paper-FAS\\Dataset\\Dataset-used\\valid\\real\\asian3_real_video_1.jpg', 'C:\\Users\\GIG

INFO:tensorflow:Assets written to: C:\Users\GIGABYTE\AppData\Local\Temp\tmpq186bx38\assets


INFO:tensorflow:Assets written to: C:\Users\GIGABYTE\AppData\Local\Temp\tmpq186bx38\assets
C:\Users\GIGABYTE\WeatherPrediction\.venv\lib\site-packages\tensorflow\lite\python\convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


{'keras': None,
 'tflite': 'C:\\Users\\GIGABYTE\\PycharmProjects\\Paper-FAS\\model2\\face_antispoofing_v2_int8.tflite',
 'keras_error': 0,
 'tflite_error': 0}

In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
from torchvision import transforms

# Pipeline inference (khớp với representative dataset)
inf_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomResizedCrop(size=224, scale=(0.9, 1.0)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x * 255)
])

# Đường dẫn file ảnh test
test_image_path = r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\Dataset\Dataset-used\test\real\asian1_real_selfie.jpg"

# Load và xử lý ảnh test
test_img = Image.open(test_image_path).convert('RGB')
test_img = inf_transform(test_img)  # (3, 224, 224)
test_np = test_img.numpy()           # (3, 224, 224)
test_np = np.transpose(test_np, (1, 2, 0))  # (224, 224, 3)
test_np = np.expand_dims(test_np, axis=0)     # (1, 224, 224, 3)
test_np = test_np.astype(np.uint8)

# Khởi tạo TFLite Interpreter và load mô hình
tflite_model_path = r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\model2\face_antispoofing_v2_int8.tflite"
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Lấy thông tin input và output
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input details:", input_details)
print("Output details:", output_details)

# Đưa dữ liệu vào mô hình và chạy inference
interpreter.set_tensor(input_details[0]['index'], test_np)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])

# Dequantize output
scale, zero_point = output_details[0]['quantization']
output_float = (output.astype(np.float32) - zero_point) * scale if scale != 0 else output
prediction = np.argmax(output_float, axis=1)[0]

print("Prediction:", prediction)
print("Output raw (dequantized):", output_float)

Input details: [{'name': 'serving_default_input_5:0', 'index': 0, 'shape': array([  1, 224, 224,   3]), 'shape_signature': array([  1, 224, 224,   3]), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.01975196972489357, 113), 'quantization_parameters': {'scales': array([0.01975197], dtype=float32), 'zero_points': array([113]), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall:0', 'index': 199, 'shape': array([1, 2]), 'shape_signature': array([1, 2]), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.18672429025173187, 127), 'quantization_parameters': {'scales': array([0.18672429], dtype=float32), 'zero_points': array([127]), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Prediction: 0
Output raw (dequantized): [[ 2.61414 -2.61414]]


In [13]:
import tensorflow as tf

# Giả sử interpreter đã được khởi tạo và allocate_tensors() được gọi
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# In thông tin chi tiết của input
for i, inp in enumerate(input_details):
    print(f"Input {i}:")
    print("  Shape:", inp['shape'])
    print("  Dtype:", inp['dtype'])
    print("  Quantization:", inp['quantization'])

# In thông tin chi tiết của output
for i, out in enumerate(output_details):
    print(f"Output {i}:")
    print("  Shape:", out['shape'])
    print("  Dtype:", out['dtype'])
    print("  Quantization:", out['quantization'])


Input 0:
  Shape: [  1 224 224   3]
  Dtype: <class 'numpy.uint8'>
  Quantization: (0.019774096086621284, 113)
Output 0:
  Shape: [1 2]
  Dtype: <class 'numpy.uint8'>
  Quantization: (0.2599039673805237, 137)


#### check tflite

In [69]:
import numpy as np
import tensorflow as tf
from PIL import Image
from torchvision import transforms

# ??nh ngh?a pipeline inference: c? th? s? d?ng Normalize n?u m? h?nh g?c ?? ???c hu?n luy?n v?i Normalize
transform_inference = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ???ng d?n ??n ?nh test
image_path = r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\Dataset\train_convert_reduced\real\cati2_real_132.jpg"
img = Image.open(image_path).convert('RGB')
img_transformed = transform_inference(img)
# Chuy?n ??i tensor (C, H, W) sang numpy (1, H, W, C)
img_np = img_transformed.numpy().transpose(1, 2, 0)
img_np = np.expand_dims(img_np, axis=0).astype(np.float32)

# T?o TFLite Interpreter v? ch?y inference
tflite_model_path = r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\Model\face_antispoofing.tflite"  # float model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], img_np)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
prediction = np.argmax(output, axis=1)[0]

print("Prediction:", prediction)
print("Output raw:", output)


Prediction: 1
Output raw: [[-7.746914   7.3939495]]


In [15]:
from torch.utils.data import DataLoader
# T?o test dataset v? loader
test_dataset = FaceDataset(root_dir=test_dir, transform=transform_inference)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [16]:
# H?m t?nh to?n c?c ch? s? t? m? h?nh TFLite
def calculate_metrics_tflite(interpreter, test_loader):
    """
    T?nh to?n ACER, APCER, BPCER t? m? h?nh TFLite float.
    Gi? s? m? h?nh c? ??u v?o shape: (batch, 224, 224, 3) v? output shape: (batch, num_classes)
    """
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    total_real = 0
    total_fake = 0
    misclassified_real = 0
    misclassified_fake = 0

    # L?p qua c?c batch trong test_loader
    for images, labels in test_loader:
        # images: tensor shape (batch, C, H, W)
        batch = images.size(0)
        # Chuy?n sang numpy v? chuy?n t? NCHW sang NHWC
        images_np = images.numpy().transpose(0, 2, 3, 1)  # shape: (batch, 224, 224, 3)

        # N?u m? h?nh TFLite h? tr? batch inference th?:
        try:
            interpreter.set_tensor(input_details[0]['index'], images_np)
            interpreter.invoke()
            outputs = interpreter.get_tensor(output_details[0]['index'])
            # outputs: shape (batch, num_classes)
            preds = np.argmax(outputs, axis=1)
        except Exception as e:
            # N?u c? l?i v?i batch inference, chuy?n sang inference t?ng ?nh
            preds = []
            for i in range(batch):
                input_data = np.expand_dims(images_np[i], axis=0).astype(np.float32)
                interpreter.set_tensor(input_details[0]['index'], input_data)
                interpreter.invoke()
                output_data = interpreter.get_tensor(output_details[0]['index'])
                preds.append(np.argmax(output_data, axis=1)[0])
            preds = np.array(preds)

        # So s?nh d? ?o?n v?i nh?n th?t
        for label, pred in zip(labels.numpy(), preds):
            if label == 0:  # fake
                total_fake += 1
                if pred != 0:
                    misclassified_fake += 1
            elif label == 1:  # real
                total_real += 1
                if pred != 1:
                    misclassified_real += 1

    apcer = misclassified_fake / total_fake if total_fake > 0 else 0
    bpcer = misclassified_real / total_real if total_real > 0 else 0
    acer = (apcer + bpcer) / 2
    return acer, apcer, bpcer

# ???ng d?n m? h?nh TFLite float (chuy?n ??i t? ONNX sang TFLite float, kh?ng quantized)
tflite_model_path = r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\Model\face_antispoofing.tflite"
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# T?nh to?n c?c ch? s?
acer, apcer, bpcer = calculate_metrics_tflite(interpreter, test_loader)
print(f"ACER: {acer:.4f}")
print(f"APCER: {apcer:.4f}")
print(f"BPCER: {bpcer:.4f}")

ACER: 0.0221
APCER: 0.0112
BPCER: 0.0329


In [ ]:
import os

def get_model_size(file_path):
    size_bytes = os.path.getsize(file_path)
    size_mb = size_bytes / (1024 * 1024)
    return size_mb

model_path = r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\Model\face_antispoofing.tflite"
size = get_model_size(model_path)
print(f"K?ch th??c m? h?nh: {size:.2f} MB")


### Chuy?n ??i xu?ng d?ng float16

In [ ]:
import sys
# Th?m th? m?c cha ch?a folder 'onnx2tflite'
sys.path.append(r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\onnx2tflite")
from onnx2tflite.converter import onnx_converter

onnx_converter(
    onnx_model_path = r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\Model\face_antispoofing.onnx",
    need_simplify = True,
    output_path = r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\Model",
    target_formats = ['tflite'],  # chuy?n ??i sang TFLite
    weight_quant = False,
    int8_model = False,    # v? hi?u h?a int8
    fp16_model = True,     # k?ch ho?t float16
)


In [67]:
import tensorflow as tf

# Giả sử interpreter đã được khởi tạo và allocate_tensors() được gọi
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# In thông tin chi tiết của input
for i, inp in enumerate(input_details):
    print(f"Input {i}:")
    print("  Shape:", inp['shape'])
    print("  Dtype:", inp['dtype'])
    print("  Quantization:", inp['quantization'])

# In thông tin chi tiết của output
for i, out in enumerate(output_details):
    print(f"Output {i}:")
    print("  Shape:", out['shape'])
    print("  Dtype:", out['dtype'])
    print("  Quantization:", out['quantization'])


Input 0:
  Shape: [  1 224 224   3]
  Dtype: <class 'numpy.uint8'>
  Quantization: (0.020829275250434875, 115)
Output 0:
  Shape: [1 2]
  Dtype: <class 'numpy.uint8'>
  Quantization: (0.6434805393218994, 122)


#### check float16

In [22]:
# H?m t?nh to?n c?c ch? s? t? m? h?nh TFLite
def calculate_metrics_float16(interpreter, test_loader):
    """
    T?nh to?n ACER, APCER, BPCER t? m? h?nh TFLite float.
    Gi? s? m? h?nh c? ??u v?o shape: (batch, 224, 224, 3) v? output shape: (batch, num_classes)
    """
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    total_real = 0
    total_fake = 0
    misclassified_real = 0
    misclassified_fake = 0

    # L?p qua c?c batch trong test_loader
    for images, labels in test_loader:
        # images: tensor shape (batch, C, H, W)
        batch = images.size(0)
        # Chuy?n sang numpy v? chuy?n t? NCHW sang NHWC
        images_np = images.numpy().transpose(0, 2, 3, 1)  # shape: (batch, 224, 224, 3)

        # N?u m? h?nh TFLite h? tr? batch inference th?:
        try:
            interpreter.set_tensor(input_details[0]['index'], images_np)
            interpreter.invoke()
            outputs = interpreter.get_tensor(output_details[0]['index'])
            # outputs: shape (batch, num_classes)
            preds = np.argmax(outputs, axis=1)
        except Exception as e:
            # N?u c? l?i v?i batch inference, chuy?n sang inference t?ng ?nh
            preds = []
            for i in range(batch):
                input_data = np.expand_dims(images_np[i], axis=0).astype(np.float32)
                interpreter.set_tensor(input_details[0]['index'], input_data)
                interpreter.invoke()
                output_data = interpreter.get_tensor(output_details[0]['index'])
                preds.append(np.argmax(output_data, axis=1)[0])
            preds = np.array(preds)

        # So s?nh d? ?o?n v?i nh?n th?t
        for label, pred in zip(labels.numpy(), preds):
            if label == 0:  # fake
                total_fake += 1
                if pred != 0:
                    misclassified_fake += 1
            elif label == 1:  # real
                total_real += 1
                if pred != 1:
                    misclassified_real += 1

    apcer = misclassified_fake / total_fake if total_fake > 0 else 0
    bpcer = misclassified_real / total_real if total_real > 0 else 0
    acer = (apcer + bpcer) / 2
    return acer, apcer, bpcer

# ???ng d?n m? h?nh TFLite float (chuy?n ??i t? ONNX sang TFLite float, kh?ng quantized)
float16_model_path = r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\Model\face_antispoofing_fp16.tflite"
interpreter = tf.lite.Interpreter(model_path=float16_model_path)
interpreter.allocate_tensors()

# T?nh to?n c?c ch? s?
acer, apcer, bpcer = calculate_metrics_float16(interpreter, test_loader)
print(f"ACER: {acer:.4f}")
print(f"APCER: {apcer:.4f}")
print(f"BPCER: {bpcer:.4f}")

ACER: 0.0200
APCER: 0.0112
BPCER: 0.0288


In [23]:
import os

def get_model_size(file_path):
    size_bytes = os.path.getsize(file_path)
    size_mb = size_bytes / (1024 * 1024)
    return size_mb

keras_model = r"C:\Users\GIGABYTE\PycharmProjects\Paper-FAS\Model\face_antispoofing_fp16.tflite"
size = get_model_size(keras_model)
print(f"K?ch th??c m? h?nh: {size:.2f} MB")


K?ch th??c m? h?nh: 11.76 MB
